# catalytic combustion of HAN and MeOH 

Notebook based on Emily's partial oxidation of methane study.

Requires these packages
```
seaborn
cantera
```

In [1]:
import sys
sys.path

['/Users/rwest/Dropbox (CoMoChEng)/Northeastern/Research-MHI/HAN/CanteraModels',
 '/Users/rwest/Code/Cat/RMG-Py',
 '/Users/rwest/Dropbox (CoMoChEng)/Northeastern/Research-MHI/HAN/CanteraModels',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python37.zip',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7/lib-dynload',
 '',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7/site-packages',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg',
 '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/rwest/.ipython']

In [2]:
"""
This example is from doi:10.1016/j.jcat.2007.05.011, Horn 2007
Ref 17 is doi:10.1016/j.cat.2006.05.008
Ref 18 is doi:10.1007/s10562-006-0117-8

surface to reactor volume ratio (assuming same active catalytic area) A/V, at 1600.0 m-1, from Ref 17
assume catalyst is a monolith, at 16.5 mm in diam, from Ref 17 & Ref 18
porosity is 0.81, from Ref 17
front heat sheild, cataylst, and back heat sheild is 10 mm long, from Ref 17
inlet mol flow at 0.208 mol/min

graphs are in:
exit conversion % v c/0 ratio
flow/mol/min v position(mm)
S at end oxidation zone/ % v C/O ratio
selectivity S, yield(Y)/% v C/O ratio
flow/ mol/min v position (z)/ mm

also had plots with flow/ mol/ min v C/O ratio w lines of thermodynamic equilibrium )ph = const)
"""

import cantera as ct
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot  as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import cm
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
import csv

import os
import rmgpy
import rmg
import re
import operator
import pandas as pd
import pylab
from cycler import cycler
import seaborn as sns
import os
import multiprocessing
import sys
from functools import partial
sys.path.append(os.getcwd())
%matplotlib inline

RDKit WARNING: [19:49:13] Enabling RDKit 2019.09.1 jupyter extensions


In [3]:
ct

<module 'cantera' from '/Users/rwest/opt/anaconda3/envs/rmg3/lib/python3.7/site-packages/cantera/__init__.py'>

In [13]:
import cantera.ctml_writer
cantera.ctml_writer.validate(species='no', reactions='no')

In [14]:
ct.ctml_writer._valrxn

'no'

In [16]:
# this chemkin file is from the cti generated by rmg
gas = ct.Solution('../RMG-model/cantera/chem_annotated.cti','gas')
surf = ct.Interface('../RMG-model/cantera/chem_annotated.cti','surface1', [gas])

print("This mechanism contains {} gas reactions and {} surface reactions".format(gas.n_reactions, surf.n_reactions))



**** WARNING ****
For species HOCHO(54), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value computed using high-temperature polynomial: -38.1678


CanteraError: 
***********************************************************************
CanteraError thrown by Plog::validate:
Invalid rate coefficient for reaction 'C2H3(47) + CH2O(13) <=> C2H4(21) + HCO(18)'
at P = 101325, T = 200
***********************************************************************


In [ ]:
i_ar = gas.species_index('Ar')
i_ch4 = gas.species_index('CH4(2)')
i_o2 = gas.species_index('O2(3)')
i_co2 = gas.species_index('CO2(4)')
i_h2o = gas.species_index('H2O(5)')
i_h2 = gas.species_index('H2(6)')
i_co = gas.species_index('CO(7)')

errors = []

# unit conversion factors to SI
mm = 0.001
cm = 0.01
ms = mm
minute = 60.0

#######################################################################
# Input Parameters
#######################################################################
t_in = 700 #K - in the paper, it was ~698.15K at the start of the cat surface and ~373.15 for the gas inlet temp
t_cat = t_in 
length = 70 * mm  # Reactor length - catalyst length 10mm, but it doesn't say where.  let's guess at 1 cm?
diam = 16.5*mm # Reactor diameter 
area = (diam/2.0)**2*np.pi  # Reactor cross section area (area of tube) in m^2
porosity = 0.81  # Monolith channel porosity, from Horn ref 17 sec 2.2.2
cat_area_per_vol = 1600.  # Catalyst particle surface area per unit volume in m-1
flow_rate = 4.7 # slpm
flow_rate = flow_rate*.001/60 # m^3/s
velocity = flow_rate/area # m/s

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 7001

on_catalyst = 1000
off_catalyst = 2000
dt = 1.0

reactor_len = length/(NReactors-1)
rvol = area * reactor_len * porosity

# catalyst area in one reactor
cat_area = cat_area_per_vol * rvol

mass_flow_rate = velocity * gas.density * area # kg/s

In [ ]:
def monolithFull(gas,surf,temp,mol_in,verbose=False,sens=False):
    """
    Verbose prints out values as you go along
    Sens is for sensitivity, in the form [perturbation, reaction #]
    """
    
    ch4,o2,ar = mol_in
    ratio = ch4/(2*o2)
    ratio = round(ratio,1)
    ratio = str(ratio)
    ch4=str(ch4);o2=str(o2);ar=str(ar)
    X = str('CH4(2):'+ch4+', O2(3):'+o2+', Ar:'+ar)
    gas.TPX = temp, ct.one_atm, X
    temp_cat = temp
    surf.TP = temp_cat, ct.one_atm
    surf.coverages = 'X(1):1.0'
    gas.set_multiplier(1.0)
    
    TDY = gas.TDY
    cov = surf.coverages

    if verbose is True:
        print('  distance(mm)   X_CH4        X_O2        X_H2       X_CO       X_H2O       X_CO2')

    # create a new reactor
    gas.TDY = TDY
    r = ct.IdealGasReactor(gas)
    r.volume = rvol

    # create a reservoir to represent the reactor immediately upstream. Note
    # that the gas object is set already to the state of the upstream reactor
    upstream = ct.Reservoir(gas, name='upstream')

    # create a reservoir for the reactor to exhaust into. The composition of
    # this reservoir is irrelevant.
    downstream = ct.Reservoir(gas, name='downstream')

    # Add the reacting surface to the reactor. The area is set to the desired
    # catalyst area in the reactor.
    rsurf = ct.ReactorSurface(surf, r, A=cat_area)

    # The mass flow rate into the reactor will be fixed by using a
    # MassFlowController object.
    m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

    # We need an outlet to the downstream reservoir. This will determine the
    # pressure in the reactor. The value of K will only affect the transient
    # pressure difference.
    v = ct.PressureController(r, downstream, master=m, K=1e-5)

    sim = ct.ReactorNet([r])
    sim.max_err_test_fails = 12

    # set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-10
    sim.atol = 1.0e-18

    gas_names = gas.species_names
    surf_names = surf.species_names
    gas_out = [] # in mol fractions
    surf_out = []
    dist_array=[]
    T_array = []


    surf.set_multiplier(0.0) #no surface reactions until the gauze
    for n in range(NReactors):
        # Set the state of the reservoir to match that of the previous reactor
        gas.TDY = r.thermo.TDY
        upstream.syncState()
        if n==on_catalyst:
            surf.set_multiplier(1.0)
            if sens is not False:
                surf.set_multiplier(1.0+sens[0],sens[1])
        if n==off_catalyst:
            surf.set_multiplier(0.0)
        sim.reinitialize()
        sim.advance_to_steady_state()
        dist = n * reactor_len * 1.0e3   # distance in mm
        dist_array.append(dist)
        T_array.append(surf.T)
        gas_out.append(gas.X.copy())
        surf_out.append(surf.X.copy())
        
        # make reaction diagrams
        out_dir = 'rxnpath'
        os.path.exists(out_dir) or os.makedirs(out_dir)
        elements = ['H', 'O']
        locations_of_interest = [1000,1200,1400,1600,1800,1999]
        for l in locations_of_interest:
            if n==l:
                location = str(int(n/100))

                diagram = ct.ReactionPathDiagram(surf,'X')
                diagram.title = 'rxn path'
                diagram.label_threshold = 1e-9
                dot_file = out_dir + '/rxnpath-' + ratio + '-x-' + location + 'mm.dot'
                img_file = out_dir + '/rxnpath-' + ratio + '-x-' + location + 'mm.png'
                img_path = os.path.join(os.getcwd(), img_file)
                diagram.write_dot(dot_file)
                os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
                  
                for element in elements:
#                     diagram = ct.ReactionPathDiagram(gas,element)
#                     diagram.title = element + 'rxn path'
#                     diagram.label_threshold = 1e-9
#                     dot_file = 'rxnpath-gas-' + location + 'mm-' + element + '.dot'
#                     img_file = 'rxnpath-gas-' + location + 'mm-' + element + '.png'
#                     img_path = os.path.join(os.getcwd(), img_file)
#                     diagram.write_dot(dot_file)
#                     os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
                    
                    diagram = ct.ReactionPathDiagram(surf,element)
                    diagram.title = element + 'rxn path'
                    diagram.label_threshold = 1e-9
                    dot_file = out_dir + '/rxnpath-' + ratio + '-surf-' + location + 'mm-' + element + '.dot'
                    img_file = out_dir + '/rxnpath-' + ratio + '-surf-' + location + 'mm-' + element + '.png'
                    img_path = os.path.join(out_dir, img_file)
                    diagram.write_dot(dot_file)
                    os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))

        if verbose is True:
            if not n % 100:
                print('  {0:10f}  {1:10f}  {2:10f}  {3:10f} {4:10f} {5:10f} {6:10f}'.format(dist, *gas['CH4(2)','O2(3)','H2(6)','CO(7)','H2O(5)','CO2(4)'].X))
                #print(surf.T)
                #print(gas.P)
                #print(surf.coverages)

    gas_out = np.array(gas_out)
    surf_out = np.array(surf_out)
    gas_names = np.array(gas_names)
    surf_names = np.array(surf_names)
    return gas_out, surf_out, gas_names, surf_names, dist_array, T_array

In [ ]:
def plotflow(a):
    gas_out, surf_out, gas_names, surf_names, dist_array, T_array = a
    gas_out = gas_out * .208
    
    # Plot in mol/min
    
    # configure the plot
    fig, axs = plt.subplots(1,2)

    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k','g']))                          

    #PLOT WITH GAS PHASE REACTIONS INCLUDED
    for i in range(len(gas_out[0,:])):
        if i != i_ar:
            if gas_out[:,i].max() > 5.e-3:
    #             print(gas_names[i])
                axs[0].plot(dist_array, gas_out[:,i], label=gas_names[i])
                species_name = gas_names[i]
                if species_name.endswith(')'):
                                if species_name[-3] == '(':
                                    species_name = species_name[0:-3]
                                else:
                                    species_name = species_name[0:-4]
                if species_name == "O2":
                    axs[0].annotate("O$_2$", fontsize=12, xy=(dist_array[3800], gas_out[:,i][3800]+gas_out[:,i][3800]/100.0), va =('bottom'), ha = ('center'))
                elif species_name == "CO2":
                    axs[0].annotate("CO$_2$", fontsize=12, xy=(dist_array[3800], gas_out[:,i][3800]+gas_out[:,i][3800]/10.0), va =('top'), ha = ('center'))
                elif species_name == "CO":
                    axs[0].annotate("CO", fontsize=12, xy=(dist_array[3800], gas_out[:,i][3800]+0.001), va =('bottom'), ha = ('center'))
                elif species_name == "CH2O":
                    axs[0].annotate("CH$_2$O", fontsize=12, xy=(dist_array[3800], gas_out[:,i][3800]+0.001), va =('bottom'), ha = ('center'))    
                elif species_name == "CH4":
                    axs[0].annotate("CH$_4$", fontsize=12, xy=(dist_array[3800], gas_out[:,i][3800]+gas_out[:,i][3800]/100.0), va =('bottom'), ha = ('center'))  
                elif species_name == "H2O":
                    axs[0].annotate("H$_2$O", fontsize=12, xy=(dist_array[3800], gas_out[:,i][3800]+gas_out[:,i][3800]/40.0), va =('top'), ha = ('center'))   
                else:                     
                    axs[0].annotate(species_name, fontsize=12, xy=(dist_array[3800], gas_out[:,i][-1]+gas_out[:,i][-1]/10.0), va =('top'), ha = ('center'))
            else:
                axs[0].plot(0,0)

    axs[1].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k','g']))  
    axs[1].plot(dist_array, T_array, label="surface + gas reactions")

    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k','g']))   

    axs[0].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[0,0.3], linestyle='--', color='xkcd:grey')
    axs[0].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[0,0.3], linestyle='--', color='xkcd:grey')  
    axs[0].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], 0.275), va =('bottom'), ha = ('left'))
    axs[1].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[400.0,1500], linestyle='--', color='xkcd:grey')
    axs[1].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[400.0,1500], linestyle='--', color='xkcd:grey')
    axs[1].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], 1265), va =('bottom'), ha = ('left'))

    for item in (axs[0].get_xticklabels() + axs[0].get_yticklabels() + axs[1].get_xticklabels() + axs[1].get_yticklabels()):
        item.set_fontsize(12)

    axs[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),fancybox=False, shadow=False, ncol=2)
    axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),fancybox=False, shadow=False, ncol=4)
    axs[0].set_ylim(0.,0.1); axs[1].set_ylim(400.0,2400)
    axs[0].set_xlim(0.0,length/mm); axs[1].set_xlim(0.0,length/mm)
    axs[0].set_xlabel('Distance (mm)', fontsize=13); axs[1].set_xlabel('Distance (mm)', fontsize=13); #axs[0,1].set_xlabel('time (s)'); axs[1,1].set_xlabel('time (s)')
    axs[0].set_ylabel('flow/ mol/min', fontsize=13); axs[1].set_ylabel('Temperature (K)', fontsize=13)
    fig.set_figheight(6)
    fig.set_figwidth(18)
    
#     temperature = np.round(T_array[0],0)
    
    for n in range(len(gas_names)):
        if gas_names[n]=='CH4(2)':
            c_in = gas_out[0][n]
        if gas_names[n]=='O2(3)':
            o_in = gas_out[0][n]
    ratio = round(c_in/(o_in*2),1)
    
    # uncomment to save figure
    out_dir = 'figures'
    os.path.exists(out_dir) or os.makedirs(out_dir)
    fig.savefig(out_dir+'/'+str(ratio)+ 'ratio.png', bbox_inches='tight')

In [ ]:
def plotZoom(a):
    gas_out, surf_out, gas_names, surf_names, dist_array, T_array = a
#     gas2_out, surf2_out, gas2_names, surf2_names, dist2_array, T2_array = b
    # configure the plot
    gas_out = gas_out*0.208
    
    fig, axs = plt.subplots(1,2)

    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k','g']))                          

    #PLOT WITH GAS PHASE REACTIONS INCLUDED
    for i in range(len(gas_out[0,:])):
        if i != i_ar:
            if gas_out[:,i].max() > 5.e-3:
    #             print(gas_names[i])
                axs[0].plot(dist_array, gas_out[:,i], label=gas_names[i])
                species_name = gas_names[i]
                if species_name.endswith(')'):
                                if species_name[-3] == '(':
                                    species_name = species_name[0:-3]
                                else:
                                    species_name = species_name[0:-4]
                if species_name == "O2":
                    axs[0].annotate("O$_2$", fontsize=12, xy=(dist_array[2200], gas_out[:,i][2200]+gas_out[:,i][2200]/100.0), va =('bottom'), ha = ('center'))
                elif species_name == "CO2":
                    axs[0].annotate("CO$_2$", fontsize=12, xy=(dist_array[2200], gas_out[:,i][2200]+gas_out[:,i][2200]/10.0), va =('top'), ha = ('center'))
                elif species_name == "CO":
                    axs[0].annotate("CO", fontsize=12, xy=(dist_array[2200], gas_out[:,i][2200]+0.001), va =('bottom'), ha = ('center'))
                elif species_name == "CH2O":
                    axs[0].annotate("CH$_2$O", fontsize=12, xy=(dist_array[2200], gas_out[:,i][2200]+0.001), va =('bottom'), ha = ('center'))    
                elif species_name == "CH4":
                    axs[0].annotate("CH$_4$", fontsize=12, xy=(dist_array[2200], gas_out[:,i][2200]+gas_out[:,i][2200]/100.0), va =('bottom'), ha = ('center'))  
                elif species_name == "H2O":
                    axs[0].annotate("H$_2$O", fontsize=12, xy=(dist_array[2200], gas_out[:,i][2200]+gas_out[:,i][2200]/40.0), va =('top'), ha = ('center'))   
                else:                     
                    axs[0].annotate(species_name, fontsize=12, xy=(dist_array[-1], gas_out[:,i][-1]+gas_out[:,i][-1]/10.0), va =('top'), ha = ('center'))
            else:
                axs[0].plot(0,0)

    axs[1].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k','g']))  
    #Plot two temperatures (of gas-phase and surface vs only surface.)
    axs[1].plot(dist_array, T_array, label="surface + gas reactions")
#     axs[1].plot(dist_array, T2_array, "--", label="surface reactions only")


    axs[0].set_prop_cycle(cycler('color', ['m', 'g', 'b', 'y', 'c', 'r', 'k','g']))    

    axs[0].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[0,0.2], linestyle='--', color='xkcd:grey')
    axs[0].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[0,0.2], linestyle='--', color='xkcd:grey')  
    axs[0].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], 0.175), va =('bottom'), ha = ('left'))
    axs[1].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[600.0,2000], linestyle='--', color='xkcd:grey')
    axs[1].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[600.0,2000], linestyle='--', color='xkcd:grey')
    axs[1].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], 1800), va =('bottom'), ha = ('left'))

    for item in (axs[0].get_xticklabels() + axs[0].get_yticklabels() + axs[1].get_xticklabels() + axs[1].get_yticklabels()):
        item.set_fontsize(12)

    axs[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),fancybox=False, shadow=False, ncol=2)
    axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),fancybox=False, shadow=False, ncol=4)
    axs[0].set_ylim(0.,0.1); axs[1].set_ylim(600.0,2000)
    axs[0].set_xlim(8,25); axs[1].set_xlim(8,25)
    axs[0].set_xlabel('Distance (mm)', fontsize=13); axs[1].set_xlabel('Distance (mm)', fontsize=13); #axs[0,1].set_xlabel('time (s)'); axs[1,1].set_xlabel('time (s)')
    axs[0].set_ylabel('flow/ mol/min', fontsize=13); axs[1].set_ylabel('Temperature (K)', fontsize=13)
    #fig.tight_layout()
    #axs[1,0].ticklabel_format(axis='x', style='sci', scilimits=(0,0))
    #axs[0,1].ticklabel_format(axis='x', style='sci', scilimits=(0,0))
    #axs[1,1].ticklabel_format(axis='x', style='sci', scilimits=(0,0))
    fig.set_figheight(6)
    fig.set_figwidth(18)

#     temperature = np.round(T_array[0],0)
    for n in range(len(gas_names)):
        if gas_names[n]=='CH4(2)':
            c_in = gas_out[0][n]
        if gas_names[n]=='O2(3)':
            o_in = gas_out[0][n]
    ratio = c_in/(o_in*2)
    ratio = round(ratio,1)
    
    # uncomment to save figure
    out_dir = 'figures'
    os.path.exists(out_dir) or os.makedirs(out_dir)
    fig.savefig(out_dir+'/'+str(ratio)+ 'ratioZoom.png', bbox_inches='tight')

In [ ]:
# at different ratios
tot_flow = 0.208 # from Horn 2007, constant inlet flow rate in mol/min, equivalent to 4.7 slpm
ratios = [.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.6,1.8,2.,2.2,2.4,2.6] # 15 items

data = []
t = t_in # temperature, in K

for r in ratios:
    # get the moles in for a certain ratio
    fo2 = tot_flow/(2.*r+1+79/21)
    fch4 = 2*fo2*r
    far = 79*fo2/21
    
    moles_in = [fch4,fo2,far]
    
    try:
        a = monolithFull(gas,surf,t,moles_in,False)
        plotflow(a)
        plotZoom(a)
        
        gas_out, surf_out, gas_names, surf_names, dist_array, T_array = a
        
        data.append([r,[gas_out,gas_names,dist_array,T_array]])
    except:
        print('Unable to run simulation at a C/O ratio of {:.0f}'.format(r))
        errors.append(r)
        pass

In [ ]:
# finding exit conversions
end_temp = []
ch4_conv = []
o2_conv = []
co_sel = []
h2_sel = []
h2o_sel = []
ratios_real = []
for r in data:
#     gas_out,gas_names,dist_array,T_array = r[1]
    for x in range(len(r[1][1])):
        if r[1][1][x]=='CH4(2)':
            ch4_in = r[1][0][0][x]
            ch4_out = r[1][0][-1][x]
            if ch4_out < 0:
                ch4_out = 0.
            elif ch4_out > ch4_in:
                ch4_out = ch4_in
            conv = (ch4_in-ch4_out)/ch4_in
            if conv < 0:
                ch4_conv.append(1e-15)
            else:
                ch4_conv.append(conv)
        if r[1][1][x]=='O2(3)':
            o2_in = r[1][0][0][x]
            o2_out = r[1][0][-1][x]
            if o2_out < 0:
                o2_out = 0.
            elif o2_out > o2_in:
                o2_out = o2_in
            conv = (o2_in-o2_out)/o2_in
            if conv < 0:
                o2_conv.append(1e-15)
            else:
                o2_conv.append(conv)
    ratios_real.append(ch4_in/(2*o2_in))
    end_temp.append(r[1][3][-1])
    
    for x in range(len(r[1][1])):
        if r[1][1][x]=='Ar':
            ar = r[1][0][-1][x]
        if r[1][1][x]=='CO(7)':
            co_out = r[1][0][-1][x]
            co_sel.append(co_out/(1-ch4_out-o2_out-ar))
        if r[1][1][x]=='H2O(5)':
            h2o_out = r[1][0][-1][x]
            h2o_sel.append(h2o_out/(1-ch4_out-o2_out-ar))
        if r[1][1][x]=='H2(6)':
            h2_out = r[1][0][-1][x]
            h2_sel.append(h2_out/(1-ch4_out-o2_out-ar))
        
#     ratios.append(r[0])

# write this metal data to a csv file for later use in the lsr volcano plot

output = []
for x in range(len(ratios_real)):
    output.append([ratios_real[x],ch4_conv[x],o2_conv[x],co_sel[x],h2_sel[x],h2o_sel[x]])
# todo: some sort of check/something to output to know that all ratios were sucessfully run
k = (pd.DataFrame.from_dict(data=output, orient='columns'))
k.columns = ['C/O ratio','CH4 Conv','O2 Conv','CO Selec','H2 Selec','H2O Selec']
k.to_csv('dict_conversions_selectivities.csv', header=True)

# def export(rxns_translated,ratio,sens_vals,sens_type=1):
#     answer = dict(zip(rxns_translated, sens_vals))
#     sorted_answer = sorted(answer.items(), key=operator.itemgetter(1), reverse=False)
#     # for x in sorted_answer:
#     #     print '%s %.4f'%(x)
    
#     # write to csv file
#     (pd.DataFrame.from_dict(data=sorted_answer, orient='columns')
#     .to_csv('dict_{}ratio_{}.csv'.format(ratio,sens_type), header=False))


fig, axs = plt.subplots(1,2)
# plot exit conversion and temp
axs[0].plot(ratios_real, ch4_conv,'bo-', label='CH4', color='green')
axs[0].plot(ratios_real, o2_conv, 'bo-',label='O2', color='blue')
ax2 = axs[0].twinx()
ax2.plot(ratios_real, end_temp, 'bo-',label='temp', color = 'red')
ax2.set_ylim(600.0,2000)

# plot exit selectivities
axs[1].plot(ratios_real, h2o_sel, 'bo-',label='H2O', color ='pink')
axs[1].plot(ratios_real, co_sel, 'bo-',label='CO', color ='purple')
axs[1].plot(ratios_real, h2_sel, 'bo-',label='H2', color ='orange')

axs[0].legend()
axs[1].legend()
axs[0].set_ylabel('Exit conversion (%)', fontsize=13);
ax2.set_ylabel('Exit temperature (K)', fontsize=13)
axs[0].set_xlabel('C/O Ratio', fontsize=13);
axs[1].set_ylabel('Exit selectivity (%)', fontsize=13)
axs[1].set_xlabel('C/O Ratio', fontsize=13)
plt.tight_layout()
fig.set_figheight(6)
fig.set_figwidth(16)

out_dir = 'figures'
os.path.exists(out_dir) or os.makedirs(out_dir)
fig.savefig(out_dir+'/'+'conversion&selectivity.png', bbox_inches='tight')

In [ ]:
temps = []
o2 = []
co = []
h2 = []
ratios = []
for r in data:
    gas_out,gas_names,dist_array,T_array = r[1]
    for x in range(len(r[1][1])):
        if r[1][1][x]=='O2(3)':
            o2.append(r[1][0][:,x])
        if r[1][1][x]=='CO(7)':
            co.append(r[1][0][:,x])
        if r[1][1][x]=='H2(6)':
            h2.append(r[1][0][:,x])
    dist_array = r[1][2]
            
    temps.append(r[1][3])
    ratios.append(r[0])



fig, axs = plt.subplots(3,1)
sns.set_palette(sns.color_palette("coolwarm",15))
# sns.set_palette(sns.cubehelix_palette(15, start=.5, rot=-1))

# plot exit conversion and temp
for r in range(len(ratios)):
    axs[0].plot(dist_array,o2[r]*.208,label=ratios[r])
    axs[1].plot(dist_array,h2[r]*.208,label=ratios[r])
    axs[2].plot(dist_array,co[r]*.208,label=ratios[r])

ax2 = axs[0].twinx()

# sns.set_palette(sns.color_palette("coolwarm",15))
# sns.set_palette(sns.cubehelix_palette(15, start=.5, rot=-1))

for r in range(len(ratios)):
    ax2.plot(dist_array,temps[r])

axs[0].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[-0.02,0.2], linestyle='--', color='xkcd:grey')
axs[0].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[-0.02,0.2], linestyle='--', color='xkcd:grey')  
axs[0].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], 0.08), va =('bottom'), ha = ('left'))
# axs[1].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[-0.02,.2], linestyle='--', color='xkcd:grey')
# axs[1].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[-.02,0.2], linestyle='--', color='xkcd:grey')
# axs[1].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], .08), va =('bottom'), ha = ('left'))
# axs[2].plot([dist_array[on_catalyst], dist_array[on_catalyst]],[-.02,.2], linestyle='--', color='xkcd:grey')
# axs[2].plot([dist_array[off_catalyst], dist_array[off_catalyst]],[-.02,.2], linestyle='--', color='xkcd:grey')
# axs[2].annotate("catalyst", fontsize=13, xy=(dist_array[on_catalyst], .08), va =('bottom'), ha = ('left'))


axs[0].legend(loc='center left')
# axs[1].legend(loc='center left')
axs[0].set_ylabel('O2 Flow (mol/mm)', fontsize=13);
ax2.set_ylabel('Temperature (K)', fontsize=13)
# ax2.legend(loc='center right')
axs[0].set_xlabel('Position (mm)', fontsize=13);
axs[1].set_ylabel('H2 Flow (mol/min)', fontsize=13)
axs[2].set_ylabel('CO Flow (mol/min)', fontsize=13)
axs[1].set_xlabel('Position (mm)', fontsize=13)
axs[2].set_xlabel('Position (mm)', fontsize=13)
# axs[0].set_title('O2');axs[1].set_title('H2');axs[2].set_title('CO')
ax2.set_ylim(200,2000)
axs[0].set_ylim(0,.1)
axs[0].set_xlim(5,25);axs[1].set_xlim(5,25);axs[2].set_xlim(5,25)
# plt.tight_layout()
fig.set_figheight(14)
fig.set_figwidth(8)

out_dir = 'figures'
os.path.exists(out_dir) or os.makedirs(out_dir)
fig.savefig(out_dir+'/'+'flows.png', bbox_inches='tight')

In [ ]:
##################
## SENSITIVITY
##################

def sensitivity(gas,surf,old_data,temp,dk):
    """
    Function to get sensitivity, but running additional simulations and comparing
    to the original simulation (data) to get a numberical value for sensitivity.
    
    old_data is an array with the original simulation output.
    
    Has multiple ways to calculate sensitivity.  You can use all at once, but was
    written so that the other ways could be commented out.
    """
    rxns = []
    sens1 = []
    sens2 = []
    sens3 = []
    
    gas_out_data,gas_names_data,dist_array_data,T_array_data = old_data
    
    reference = []
    for a in range(len(gas_names_data)):
        reference.append([gas_names_data[a],[gas_out_data[:,a]]])
        
    # getting the ratio
    for x in reference:
        if x[0]=='CH4(2)':
            ch4_in = x[1][0][0]
        if x[0]=='O2(3)':
            o2_in = x[1][0][0]
        if x[0]=='Ar':
            ar_in = x[1][0][0]
    ratio = ch4_in/(2*o2_in)
    moles_in = [ch4_in, o2_in, ar_in]
    
    #####################################
    ## Sensitivity definition 1:
    ##
    ## selectivity of h2 and co added together
    ## to give a syngas selectivity
    ## 
    ## negative sensitivity is higher selectivity
    #####################################
    for x in reference:
        if x[0]=='CH4(2)':
            ch4_in = x[1][0][0]
            ch4_out = x[1][0][-1]
            if ch4_out < 0:
                ch4_out = 0.
            elif ch4_out > ch4_in:
                ch4_out = ch4_in
            conv = (ch4_in-ch4_out)/(ch4_in)
            if conv < 0:
                reference_ch4_conv = 1e-15
            else:
                reference_ch4_conv = conv
        if x[0]=='Ar':
            ar = x[1][0][-1]
        if x[0]=='O2(3)':
            o2_out = x[1][0][-1]
            if o2_out < 0:
                o2_out = 0.
            elif o2_out > o2_in:
                o2_out = o2_in
        if x[0]=='CO(7)':
            co_out = x[1][0][-1]
            reference_co_sel = co_out/(1-ch4_out-o2_out-ar)
        if x[0]=='H2(6)':
            h2_out = x[1][0][-1]
            reference_h2_sel = h2_out/(1-ch4_out-o2_out-ar)
    reference_syngas_selectivity = reference_co_sel+reference_h2_sel
    
    #####################################
    ## Sensitivity definition 2:
    ##
    ## defining by syngas selectivity multiplied
    ## by ch4 conversion to 'normalize'
    ##
    #####################################
    reference_norm_syngas_selectivity = reference_syngas_selectivity*reference_ch4_conv
    
    #####################################
    ## Sensitivity definition 3:
    ##
    ## defining by normalized syngas selectivity
    ## multiplied by syngas selectivity
    ##
    #####################################
    reference_norm_syngas_selectivity2 = reference_norm_syngas_selectivity*reference_syngas_selectivity
    
    # run the simulations
    for rxn in range(surf.n_reactions):
        c = monolithFull(gas,surf,temp,moles_in,sens=[dk,rxn])
        gas_out, surf_out, gas_names, surf_names, dist_array, T_array = c
        
        # uncomment if sensitivity for surface only reactions are wanted
#         d = monolithNoGas(gas,surf,temp,sens=[dk,rxn])
#         gas2_out, surf2_out, gas2_names, surf2_names, dist2_array, T2_array = d
        
        new_amts = []
        for a in range(len(gas_names)):
            new_amts.append([gas_names[a],[gas_out[:,a]]])
        
        #####################################
        ## Sensitivity definition 1:
        #####################################
        for x in new_amts:
            if x[0]=='CH4(2)':
                new_ch4_in = x[1][0][0]
                new_ch4_out = x[1][0][-1]
                if new_ch4_out < 0:
                    new_ch4_out = 0.
                elif new_ch4_out > new_ch4_in:
                    new_ch4_out = new_ch4_in
                new_ch4_conv = (new_ch4_in-new_ch4_out)/new_ch4_in
                if new_ch4_conv < 0:
                    new_ch4_conv = 1e-15
            if x[0]=='Ar':
                ar = x[1][0][-1]
            if x[0]=='O2(3)':
                new_o2_in = c[1][0][0]
                new_o2_out = x[1][0][-1]
                if new_o2_out < 0:
                    new_o2_out = 0.
                elif new_o2_out > new_o2_in:
                    new_o2_out = new_o2_in
            if x[0]=='CO(7)':
                new_co_out = x[1][0][-1]
                new_co_sel = new_co_out/(1-new_o2_out-new_ch4_out-ar)
            if x[0]=='H2(6)':
                new_h2_out = x[1][0][-1]
                new_h2_sel = new_h2_out/(1-new_o2_out-new_ch4_out-ar)
        new_syngas_selectivity = new_co_sel+new_h2_sel
        Sens1 = (reference_syngas_selectivity-new_syngas_selectivity)/(reference_syngas_selectivity*dk)
        sens1.append(Sens1)
#         print "%d %s %.3F"%(rxn,surf.reaction_equations()[rxn],Sens1)
        
        #####################################
        ## Sensitivity definition 2:
        #####################################
        new_norm_syngas_selectivity = new_syngas_selectivity*new_ch4_conv
        Sens2 = (reference_norm_syngas_selectivity-new_norm_syngas_selectivity)/(reference_norm_syngas_selectivity*dk)
        sens2.append(Sens2)
#         print "%d %s %.3F"%(rxn,surf.reaction_equations()[rxn],Sens2)

        #####################################
        ## Sensitivity definition 3:
        #####################################
        new_norm_syngas_selectivity2 = new_norm_syngas_selectivity*new_syngas_selectivity
        Sens3 = (reference_norm_syngas_selectivity2-new_norm_syngas_selectivity2)/(reference_norm_syngas_selectivity2*dk)
        sens3.append(Sens3)
#         print "%d %s %.3F"%(rxn,surf.reaction_equations()[rxn],Sens3)
        
        print "%d %s %.3F %.3F %.3F"%(rxn,surf.reaction_equations()[rxn],Sens1,Sens2,Sens3)

        rxns.append(surf.reaction_equations()[rxn])
    return rxns,sens1,sens2,sens3

def sensitivityWorker(gas, surf, data, temp, dk):
    # data is the worker input
    try:
        reactions, sensitivity1, sensitivity2, sensitivity3 = sensitivity(gas, surf, data[1], temp, dk)
        sensitivities = [data[0], sensitivity1, sensitivity2, sensitivity3]
        return sensitivities
    except:
        print('Unable to run sensitivity simulation at a C/O ratio of {:.1f}'.format(data[0]))
        errors.append(r)
        pass

In [ ]:
# # Set the value of the perturbation
# dk = 1.0e-2

# # this takes the longest
# sens_by_ratio = []
# for r in range(len(data)):
#     print data[r][0]
#     try:
#         reactions,sensitivity1,sensitivity2,sensitivity3=sensitivity(gas,surf,data[r][1],t_in,dk)
#         sens_by_ratio.append([data[r][0],sensitivity1,sensitivity2,sensitivity3])
#     except:
#         print('Unable to run sensitivity simulation at a C/O ratio of {:.1f}'.format(data[r][0]))
#         errors.append(r)
#         pass
    
# def parallelize():
#     pool = multiprocessing.Pool(processes=1)
#     return pool.map(sensitivity,)

sens_by_ratio = []
dk = 1.0e-2
num_threads = len(data)
pool = multiprocessing.Pool(processes = num_threads)

worker_input = []
for r in range(len(data)):
    worker_input.append([data[r][0],[data[r][1]]])

sens_by_ratio = pool.map(partial(sensitivityWorker,gas=gas,surf=surf,temp=t_in,dk=dk),worker_input)
pool.close()
pool.join()

In [ ]:
species_dict = rmgpy.data.kinetics.KineticsLibrary().getSpecies('species_dictionary.txt')
keys = species_dict.keys()

# get the first listed smiles string for each molecule
smile = []
for s in species_dict:
    smile.append(species_dict[s].molecule[0])
    if len(species_dict[s].molecule) is not 1:
        print 'There are %d dupllicate smiles for %s:'%(len(species_dict[s].molecule),s)
        for a in range(len(species_dict[s].molecule)):
            print '%s'%(species_dict[s].molecule[a])
            
# translate the molecules from above into just smiles strings
smiles = []
for s in smile:
    smiles.append(s.toSMILES())
    
names = dict(zip(keys,smiles))

rxns_translated = []

# now replace each molecule with smiles
for x in reactions:
    for key, smile in names.iteritems():
        x = re.sub(re.escape(key), smile, x)
    rxns_translated.append(x)

def export(rxns_translated,ratio,sens_vals,sens_type=1):
    answer = dict(zip(rxns_translated, sens_vals))
    sorted_answer = sorted(answer.items(), key=operator.itemgetter(1), reverse=False)
    
    # write to csv file
    out_dir = 'sensitivities'
    os.path.exists(out_dir) or os.makedirs(out_dir)
    (pd.DataFrame.from_dict(data=sorted_answer, orient='columns')
    .to_csv(outdir + '/dict_{}ratio_{}.csv'.format(ratio,sens_type), header=False))

In [ ]:
# export everything
for x in sens_by_ratio:
    for s in range(len(x)-1):
        export(rxns_translated,x[0],x[s+1],s+1)

In [ ]:
# if there are errors in generating models, print to a separate text file what they are
if len(errors)==0:
    try:
        os.remove('errors.txt')
    except:
        pass
if len(errors) != 0:
    with open('errors.txt','w') as f:
        f.write(str('Error at ratios: '))
        for x in range(len(errors)):
            f.write(str(errors[x]))
            if x != len(errors)-1:
                f.write(str(', '))

# Batch reactor

plt.semilogy(surf.reverse_rates_of_progress,'.')

for i in surf.reverse_rates_of_progress.argsort()[:-5:-1]:
    print i, surf.reaction_equation(i)